## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

# Chapter 004 Example 023C - AND Gate

In [1]:
import numpy as np

def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1.0-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1.0 - x**2

def step(x):
    if x >= 0.5:
        return 1
    else:
        return 0


class NeuralNetwork:

    def __init__(self, layers, activation='tanh'):
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_prime
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_prime

        # Set weights
        self.weights = []
        # layers = [2,2,1]
        # range of weight values (-1,1)
        # input and hidden layers - random((2+1, 2+1)) : 3 x 3
        for i in range(1, len(layers) - 1):
            r = 2*np.random.random((layers[i-1] + 1, layers[i] + 1)) -1
            self.weights.append(r)
        # output layer - random((2+1, 1)) : 3 x 1
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1
        self.weights.append(r)

    def fit(self, X, y, learning_rate=0.01, epochs=100000):
        # Add column of ones to X
        # This is to add the bias unit to the input layer
        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)
         
        for k in range(epochs):
            if k % 10000 == 0: print ('epochs:', str(k).zfill(6))
            
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.activation(dot_value)
                    a.append(activation)
            # output layer
            error = y[i] - a[-1]
            deltas = [error * self.activation_prime(a[-1])]

            # we need to begin at the second to last layer 
            # (a layer before the output layer)
            for l in range(len(a) - 2, 0, -1): 
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_prime(a[l]))

            # reverse
            # [level3(output)->level2(hidden)]  => [level2(hidden)->level3(output)]
            deltas.reverse()

            # backpropagation
            # 1. Multiply its output delta and input activation 
            #    to get the gradient of the weight.
            # 2. Subtract a ratio (percentage) of the gradient from the weight.
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)

    def predict(self, x): 
        a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)      
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        return a

In [2]:
if __name__ == '__main__':
    nf = [2,4,1]
    nn = NeuralNetwork(nf)

    X = np.array(
            [
            [0, 0],
            [0, 1],
            [1, 0],
            [1, 1]
            ]
            )

    y = np.array([0, 0, 0, 1])
    
    print('------------------------------------')
    print('Run AND Gate Network:', nf)
    print('------------------------------------')
    nn.fit(X, y)
    print('------------------------------------')
    print('Results:')
    print('------------------------------------')
    for i in range(X.shape[0]):
        e=X[i]
        p=nn.predict(e)
        pr=round(p[0],6)
        yp=step(pr)
        print('[%1d,%1d] p=%9.6f y(p)=%1d y(t)=%1d' % (e[0],e[1],pr,yp,y[i]))

------------------------------------
Run AND Gate Network: [2, 4, 1]
------------------------------------
epochs: 000000
epochs: 010000
epochs: 020000
epochs: 030000
epochs: 040000
epochs: 050000
epochs: 060000
epochs: 070000
epochs: 080000
epochs: 090000
------------------------------------
Results:
------------------------------------
[0,0] p=-0.000501 y(p)=0 y(t)=0
[0,1] p= 0.000569 y(p)=0 y(t)=0
[1,0] p= 0.000778 y(p)=0 y(t)=0
[1,1] p= 0.976916 y(p)=1 y(t)=1


In [3]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))

Done! 2019-10-19 17:34:29.410457
